In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR


import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import os


# 1. Load FAO Food Price Index (FPI)

fao_path = "./Food_price_indices_data_may629 (2).xls"  # Replace with your path
fao_df = pd.read_excel(fao_path, sheet_name=0, skiprows=2)

# Keep only columns that are not 'Unnamed'
fao_columns = [c for c in fao_df.columns if 'Unnamed' not in c]
fao_df = fao_df[fao_columns]

# Melt to long format
fao_long = fao_df.melt(
    id_vars=['Date'],
    var_name='Commodity',
    value_name='FPI'
)

# Add Year and Month
fao_long['Year'] = fao_long['Date'].dt.year
fao_long['Month'] = fao_long['Date'].dt.month

print("FAO FPI long shape:", fao_long.shape)
fao_long.head()

# 2. Load FAO monthly Food CPI (.dta)

fcpi_path = "./Inflation-data/fcpi_m.dta"  # Replace with your path
fcpi_df = pd.read_stata(fcpi_path)

# Keep only Country + numeric monthly columns (ignore _Note)
fcpi_df = fcpi_df[['Country'] + [c for c in fcpi_df.columns if c.startswith('_') and c[1:].isdigit()]]

# Melt wide to long
fcpi_long = fcpi_df.melt(
    id_vars=['Country'],
    var_name='YearMonth',
    value_name='Food_CPI'
)

# Convert YearMonth to datetime
fcpi_long['YearMonth'] = fcpi_long['YearMonth'].str.replace('_','')
fcpi_long['Date'] = pd.to_datetime(fcpi_long['YearMonth'], format='%Y%m')
fcpi_long['Year'] = fcpi_long['Date'].dt.year
fcpi_long['Month'] = fcpi_long['Date'].dt.month
fcpi_long.drop(columns=['YearMonth'], inplace=True)

print("FAO Food CPI long shape:", fcpi_long.shape)
fcpi_long.head()


# 3. Load World Bank CPI

cpi_path = "./API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_23195 (1)/API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_23195.csv"
cpi_df = pd.read_csv(cpi_path, skiprows=4)

# Melt wide to long
cpi_long = cpi_df.melt(
    id_vars=['Country Name','Country Code','Indicator Name','Indicator Code'],
    var_name='Year',
    value_name='CPI'
)

# Keep only numeric years
cpi_long = cpi_long[cpi_long['Year'].str.isnumeric()]
cpi_long['Year'] = cpi_long['Year'].astype(int)
cpi_long.dropna(subset=['CPI'], inplace=True)

print("World Bank CPI long shape:", cpi_long.shape)
cpi_long.head()


# 4. Load World Bank GDP Deflator

gdp_path = "./API_NY.GDP.DEFL.KD.ZG_DS2_en_csv_v2_22672 (1)/API_NY.GDP.DEFL.KD.ZG_DS2_en_csv_v2_22672.csv"
gdp_df = pd.read_csv(gdp_path, skiprows=4)

# Melt wide to long
gdp_long = gdp_df.melt(
    id_vars=['Country Name','Country Code','Indicator Name','Indicator Code'],
    var_name='Year',
    value_name='GDP_Deflator'
)

# Keep only numeric years
gdp_long = gdp_long[gdp_long['Year'].str.isnumeric()]
gdp_long['Year'] = gdp_long['Year'].astype(int)
gdp_long.dropna(subset=['GDP_Deflator'], inplace=True)

print("World Bank GDP Deflator long shape:", gdp_long.shape)
gdp_long.head()


# 5. Merge datasets

# Merge FAO FPI with FAO CPI on Year and Month
merged = pd.merge(
    fao_long,
    fcpi_long[['Country','Year','Month','Food_CPI']],
    on=['Year','Month'],
    how='left'
)

# Merge with World Bank CPI on Year
merged = pd.merge(
    merged,
    cpi_long[['Country Name','Year','CPI']],
    left_on=['Country','Year'],
    right_on=['Country Name','Year'],
    how='left'
)

# Merge with GDP Deflator on Year
merged = pd.merge(
    merged,
    gdp_long[['Country Name','Year','GDP_Deflator']],
    left_on=['Country','Year'],
    right_on=['Country Name','Year'],
    how='left'
)

# Drop redundant columns
merged.drop(columns=['Country Name_x','Country Name_y'], inplace=True, errors='ignore')

print("Merged dataset shape:", merged.shape)
merged.head()


In [ ]:
merged.info()

In [ ]:
merged.isnull().sum()

In [ ]:

# Count of unique countries
num_countries = merged['Country'].nunique()
print("Number of unique countries:", num_countries)

In [ ]:
# List all unique countries
unique_countries = merged['Country'].unique()
print("Countries in dataset:", unique_countries)


In [ ]:
# Filter dataset for Kenya
kenya_data = merged[merged['Country'] == 'Kenya']

# Show first few rows
display(kenya_data.head())

# Number of rows for Kenya
print("Number of rows for Kenya:", len(kenya_data))


In [ ]:
merged.describe()

# EDA

In [ ]:
# 1. Numerical features

numerical_cols = ['FPI', 'Food_CPI', 'CPI', 'GDP_Deflator']
eda_df = merged[numerical_cols].copy()

In [ ]:
# Set plot style
sns.set(style="whitegrid", palette="muted", font_scale=1.1)
plt.rcParams['figure.figsize'] = (12,6)

In [ ]:
# 2. Heatmap of correlations

plt.figure(figsize=(8,6))
corr = eda_df.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
# 3. Histograms 

eda_df.hist(bins=30, figsize=(12,8), color='skyblue', edgecolor='black')
plt.suptitle("Distribution of Numerical Features")
plt.show()

In [ ]:
# 4. Boxplots for outliers

plt.figure(figsize=(12,6))
eda_df_melt = eda_df.melt(var_name='Feature', value_name='Value')
sns.boxplot(x='Feature', y='Value', data=eda_df_melt)
plt.title("Boxplots of Numerical Features (Outlier Detection)")
plt.show()

In [ ]:
# 5. Pairplot (sample for performance)

sample_df = eda_df.sample(n=5000, random_state=42)  # sample to speed up plotting
sns.pairplot(sample_df)
plt.suptitle("Pairplot of Numerical Features (Sample)", y=1.02)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Aggregate per year & commodity
agg = merged.groupby(['Year','Commodity']).agg({
    'FPI':'mean',
    'Food_CPI':'mean',
    'CPI':'mean'
}).reset_index()


In [ ]:
# Plot FPI trends per commodity
plt.figure(figsize=(16,8))
sns.lineplot(data=agg, x='Year', y='FPI', hue='Commodity', marker='o')
plt.title('Global Food Price Index (FPI) Trends per Commodity')
plt.ylabel('FPI')
plt.xlabel('Year')
plt.legend(title='Commodity', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
#  plot Food_CPI and CPI per commodity (only for countries with Food_CPI)
plt.figure(figsize=(16,8))
sns.lineplot(data=agg, x='Year', y='Food_CPI', hue='Commodity', marker='o')
plt.title('Average Food_CPI per Commodity Across Countries')
plt.ylabel('Food_CPI')
plt.xlabel('Year')
plt.legend(title='Commodity', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
